# Libraries used

Running Kernel3.9.13 base anaconda

pip install squarify
pip install yellowbrick
pip install plotly
pip install seaborn
pip install lazypredict
pip install pandas_profiling

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns  
import matplotlib.pyplot as plt
import plotly.io as pio
#import squarify #treemap
import os
import matplotlib
import warnings

from imblearn.over_sampling import SMOTE


#to enable the inline plotting
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore')

sns.set_style("darkgrid")

In [2]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import classification_report

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
#rfc using regularization and gridsearch to find the best parameters for the model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import normaltest

from pandas_profiling import ProfileReport

from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import ClassPredictionError
from yellowbrick.style.palettes import PALETTES, SEQUENCES, color_palette

import lazypredict
from lazypredict.Supervised import LazyClassifier


warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# Function for EDA. Using the display() function to have  well-formatted tables. We are mainly using pandas to explore the datasets

def dataset_description(df_target):

    print('This is the Dataset shape: %s\n' % (df_target.shape, ))
    print('Dataset columns: %s\n' % df_target.columns)

    print('\nColumns description:\n')
    display(df_target.info())
    display(df_target.describe())  # describe the dataset

    print('\nNull values:\n')
    display(df_target.isnull().sum())  # Identify null values

#function performing a quick check on df_inspection to have best of pandas functions separated by a line
def quick_check(dataframe):
    print('First 5 rows %s\n')
    print(dataframe.head(2))
    print("=====================================")
    print('Dataframe shape %s\n')
    print(dataframe.shape)
    print("=====================================")
    print('Dataframe describe categorical %s\n')
    print(dataframe.describe(include=['O']))
    print("=====================================")
    print('Dataframe null values %s\n')
    print(dataframe.isnull().sum())
    print("=====================================")
    print('Dataframe value counts %s\n')
    print(dataframe.value_counts())
    print("=====================================")

#stats function
def stats(dataframe):
    print('Dataframe correlation %s\n')
    print(dataframe.corr())
    print("=====================================")
    print('Dataframe covariance %s\n')
    print(dataframe.cov())
    print("=====================================")
    print('Dataframe skew %s\n')
    print(dataframe.skew())
    print("=====================================")
    print('Dataframe kurtosis %s\n')
    print(dataframe.kurt())
    print("=====================================")

#create a function to normalize characters from a dataset's column in Spanish
def normalize_characters(df, column):
    df[column] = df[column].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    df[column] = df[column].str.lower()
    df[column] = df[column].str.replace('á', 'a')
    df[column] = df[column].str.replace('é', 'e')
    df[column] = df[column].str.replace('í', 'i')
    df[column] = df[column].str.replace('ó', 'o')
    df[column] = df[column].str.replace('ú', 'u')
    df[column] = df[column].str.replace('ñ', 'n')
    df[column] = df[column].str.replace('ü', 'u')
    df[column] = df[column].str.replace('ç', 'c')
    df[column] = df[column].str.replace('(', '')
    df[column] = df[column].str.replace(')', '')
    df[column] = df[column].str.replace('\'', '')
    df[column] = df[column].str.replace('´', '')
    df[column] = df[column].str.replace('`', '')
    df[column] = df[column].str.replace('’', '')
    return df.head(2)

#create function to change detypes in64 to int32 in a df
def change_dtypes(df):
    for col in df.columns:
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
        elif df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
    return df

def outlier_function(df, col_name):
    """ this function detects first and third quartile and interquartile range for a given column of a dataframe
    then calculates upper and lower limits to determine outliers conservatively
    returns the number of lower and uper limit and number of outliers respectively"""
    first_quartile = np.percentile(np.array(df[col_name].tolist()), 25)
    third_quartile = np.percentile(np.array(df[col_name].tolist()), 75)
    IQR = third_quartile - first_quartile
                        
    upper_limit = third_quartile+(3*IQR)
    lower_limit = first_quartile-(3*IQR)
    outlier_count = 0
                    
    for value in df[col_name].tolist():
        if (value < lower_limit) | (value > upper_limit):
            outlier_count +=1
        else:
            pass
    return lower_limit, upper_limit, outlier_count

In [4]:
#show all print outputs when using a function
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#display all columns
pd.set_option('display.max_columns', None)


# Importing previous dataset

In [5]:
#point to the folder where the data is stored
os.chdir(r"C:\Users\pedro\datathon")

# Loading combined_mod dataset
submission_sample_prepared2 = pd.read_csv('submission_sample_prepared.csv')

In [6]:
submission_sample_prepared2.head(1)

,PipeId,Incidence,Province,Town,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,connection_bool,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene,Diameter2,Length2,Pressure2,Average yearly temperature (°C),Min. Temperature (°C),Max. Temperature (°C),Yearly Rainfall (mm),Average year Humidity (%),Rainy days per year (days),Yearly Sun Hours (hours),Average_MonthsLastRev,InspectionYear,MonthsLastRev,No_Inspections,No_Incidences_Total,Age_pipe_at_inspection,relative_risk,preventive_maintenance_rate,Probability_rate
0,446859944,0,valencia,Alginet,2012,0.11,44.29,0.15,0,0,0,60.39,0.73,15.30,0,0.00,0,1,0,0,0,1,0,0.01,1961.25,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20,20.75,2021.00,24.00,5.00,0.00,9.00,0.00,0.00,0.00


In [7]:
submission_sample_prepared2.columns

Index(['PipeId', 'Incidence', 'Province', 'Town', 'YearBuilt', 'Diameter',
       'Length', 'Pressure', 'NumConnections', 'NumConnectionsUnder',
       'BoolBridle', 'aspect', 'Relative_Thickness', 'pipe_area',
       'Total_Connections', 'area_connection', 'connection_bool',
       'gas_natural', 'Material_Acrylonitrile-Butadiene-Styrene',
       'Material_Copper', 'Material_Fiberglass-Reinforced Plastic',
       'Material_Polyethylene', 'Material_Polypropylene', 'Diameter2',
       'Length2', 'Pressure2', 'Average yearly temperature (°C)',
       'Min. Temperature (°C)', 'Max. Temperature (°C)',
       'Yearly Rainfall (mm)', 'Average year Humidity (%)',
       'Rainy days per year (days)', 'Yearly Sun Hours (hours)',
       'Average_MonthsLastRev', 'InspectionYear', 'MonthsLastRev',
       'No_Inspections', 'No_Incidences_Total', 'Age_pipe_at_inspection',
       'relative_risk', 'preventive_maintenance_rate', 'Probability_rate'],
      dtype='object')

In [8]:
submission_sample_prepared2.shape

(909733, 42)

final shape after taking care of few outliers.

2 million less

# Merging another dataset

In [9]:
#point to the folder where the data is stored for Pedro
os.chdir(r"C:\Users\pedro\datathon\base\complementary_datasets")

#Loading sample_submission
density = pd.read_excel('population_density_province.xlsx')

In [10]:
density.head(1)

,Province,Area (sq km),Year,Population,Population density (persons/sqkm)
0,Albacete,14926,2022,385727,25.84


In [11]:
normalize_characters(density, 'Province')

,Province,Area (sq km),Year,Population,Population density (persons/sqkm)
0,albacete,14926,2022,385727,25.84
1,albacete,14926,2021,386464,25.89


In [12]:
#join density to df_baseline on Province taking only rows with year 2020
submission_sample_prepared2 = submission_sample_prepared2.merge(density[density['Year'] == 2020], on='Province', how='left')
submission_sample_prepared2.head(1)

,PipeId,Incidence,Province,Town,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,connection_bool,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene,Diameter2,Length2,Pressure2,Average yearly temperature (°C),Min. Temperature (°C),Max. Temperature (°C),Yearly Rainfall (mm),Average year Humidity (%),Rainy days per year (days),Yearly Sun Hours (hours),Average_MonthsLastRev,InspectionYear,MonthsLastRev,No_Inspections,No_Incidences_Total,Age_pipe_at_inspection,relative_risk,preventive_maintenance_rate,Probability_rate,Area (sq km),Year,Population,Population density (persons/sqkm)
0,446859944,0,valencia,Alginet,2012,0.11,44.29,0.15,0,0,0,60.39,0.73,15.30,0,0.00,0,1,0,0,0,1,0,0.01,1961.25,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20,20.75,2021.00,24.00,5.00,0.00,9.00,0.00,0.00,0.00,10807,2020,2591875,239.83


In [13]:
#compare array for column Province in density and df_baseline and compute set difference
np.setdiff1d(density['Province'].unique(), submission_sample_prepared2['Province'].unique())


array([], dtype=object)

In [14]:
del density

In [15]:
#dropping Year, Population, Area (sq km) from df_baseline
submission_sample_prepared2 = submission_sample_prepared2.drop(['Year', 'Population', 'Area (sq km)'], axis=1)

# Correlation Matrix

In [16]:
#correlation matrix for df_baseline with threshold of 0.75
corr_matrix = submission_sample_prepared2.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.75)]
to_drop


['pipe_area',
 'Total_Connections',
 'Diameter2',
 'Pressure2',
 'Min. Temperature (°C)',
 'Average year Humidity (%)',
 'Rainy days per year (days)',
 'Yearly Sun Hours (hours)',
 'MonthsLastRev',
 'Age_pipe_at_inspection',
 'relative_risk',
 'preventive_maintenance_rate',
 'Probability_rate']

In [17]:
submission_sample_prepared2.columns


Index(['PipeId', 'Incidence', 'Province', 'Town', 'YearBuilt', 'Diameter',
       'Length', 'Pressure', 'NumConnections', 'NumConnectionsUnder',
       'BoolBridle', 'aspect', 'Relative_Thickness', 'pipe_area',
       'Total_Connections', 'area_connection', 'connection_bool',
       'gas_natural', 'Material_Acrylonitrile-Butadiene-Styrene',
       'Material_Copper', 'Material_Fiberglass-Reinforced Plastic',
       'Material_Polyethylene', 'Material_Polypropylene', 'Diameter2',
       'Length2', 'Pressure2', 'Average yearly temperature (°C)',
       'Min. Temperature (°C)', 'Max. Temperature (°C)',
       'Yearly Rainfall (mm)', 'Average year Humidity (%)',
       'Rainy days per year (days)', 'Yearly Sun Hours (hours)',
       'Average_MonthsLastRev', 'InspectionYear', 'MonthsLastRev',
       'No_Inspections', 'No_Incidences_Total', 'Age_pipe_at_inspection',
       'relative_risk', 'preventive_maintenance_rate', 'Probability_rate',
       'Population density (persons/sqkm)'],
      dty

In [18]:
#drop columns
submission_sample_prepared2 = submission_sample_prepared2.drop(['InspectionYear', 'Province','Town', 'YearBuilt','relative_risk', 'preventive_maintenance_rate', 'Probability_rate',
       'pipe_area','Total_Connections', 'connection_bool','Diameter2', 'Length2', 'Pressure2', 'Average yearly temperature (°C)','Min. Temperature (°C)', 'Max. Temperature (°C)',
       'Yearly Rainfall (mm)', 'Average year Humidity (%)','Rainy days per year (days)'], axis=1)

In [19]:
submission_sample_prepared2.shape

(909733, 24)

In [20]:
submission_sample_prepared2.columns 

Index(['PipeId', 'Incidence', 'Diameter', 'Length', 'Pressure',
       'NumConnections', 'NumConnectionsUnder', 'BoolBridle', 'aspect',
       'Relative_Thickness', 'area_connection', 'gas_natural',
       'Material_Acrylonitrile-Butadiene-Styrene', 'Material_Copper',
       'Material_Fiberglass-Reinforced Plastic', 'Material_Polyethylene',
       'Material_Polypropylene', 'Yearly Sun Hours (hours)',
       'Average_MonthsLastRev', 'MonthsLastRev', 'No_Inspections',
       'No_Incidences_Total', 'Age_pipe_at_inspection',
       'Population density (persons/sqkm)'],
      dtype='object')

In [21]:
#drop 'NumConnectionsUnder', 'BoolBridle'
submission_sample_prepared2 = submission_sample_prepared2.drop(['NumConnectionsUnder', 'BoolBridle'], axis=1)

# Standardize dataframe

In [22]:
submission_sample_prepared2.head(1)

,PipeId,Incidence,Diameter,Length,Pressure,NumConnections,aspect,Relative_Thickness,area_connection,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene,Yearly Sun Hours (hours),Average_MonthsLastRev,MonthsLastRev,No_Inspections,No_Incidences_Total,Age_pipe_at_inspection,Population density (persons/sqkm)
0,446859944,0,0.11,44.29,0.15,0,60.39,0.73,0.00,1,0,0,0,1,0,113.20,20.75,24.00,5.00,0.00,9.00,239.83


In [ ]:
submission_sample_prepared2.columns

In [24]:
#Use minmax scaler to scale these columns in df_baseline: 'MonthsLastRev', 'Diameter', 'Length','Pressure', 'NumConnections', 'No_Inspections', 'No_Incidences_Total','Average_MonthsLastRev', 'Age_pipe_at_inspection', 'aspect','Relative_Thickness', 'area_connection', 'gas_natural', 'Yearly Sun Hours (hours)','Population density (persons/sqkm)'
scaler = MinMaxScaler()
submission_sample_prepared2[['MonthsLastRev', 'Diameter', 'Length','Pressure', 'NumConnections', 'No_Inspections', 'No_Incidences_Total',
'Average_MonthsLastRev', 'Age_pipe_at_inspection', 'aspect','Relative_Thickness', 'area_connection', 'gas_natural', 'Yearly Sun Hours (hours)',
'Population density (persons/sqkm)']] = scaler.fit_transform(submission_sample_prepared2[['MonthsLastRev', 'Diameter', 'Length','Pressure', 'NumConnections', 
'No_Inspections', 'No_Incidences_Total','Average_MonthsLastRev', 'Age_pipe_at_inspection', 'aspect','Relative_Thickness', 'area_connection', 'gas_natural', 'Yearly Sun Hours (hours)','Population density (persons/sqkm)']])
submission_sample_prepared2.head(1)




,PipeId,Incidence,Diameter,Length,Pressure,NumConnections,aspect,Relative_Thickness,area_connection,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene,Yearly Sun Hours (hours),Average_MonthsLastRev,MonthsLastRev,No_Inspections,No_Incidences_Total,Age_pipe_at_inspection,Population density (persons/sqkm)
0,446859944,0,0.17,0.01,0.00,0.00,0.00,0.03,0.00,1.00,0,0,0,1,0,0.83,0.75,0.76,0.36,0.00,0.26,0.28


# Pandas Profiling Report

In [ ]:
#pandas profiling report for df_baseline
#profile = ProfileReport(df_baseline, title='Pandas Profiling Report', html={'style':{'full_width':True}})
#profile.to_file('df_baseline.html')


# Exporting file for notebook 4

In [26]:
#exporting df_baseline to csv
os.chdir(r"C:\Users\pedro\datathon")
submission_sample_prepared2.to_csv('submission_sample_prepared2.csv', index=False)
